# Tutorial 1.1: Setup and Introduction

## Getting Started with GenAI and MLflow

Welcome to the first notebook in our MLflow GenAI tutorial series! This notebook will guide you through setting up your environment and understanding the MLflow platform for GenAI development.

### What You'll Learn
- What MLflow is and why it's valuable for GenAI development
- Core MLflow concepts and architecture
- How to install and configure MLflow
- How to start and access the MLflow UI

### Prerequisites
- Python 3.8 or higher
- pip package manager
- OpenAI API key (or other LLM provider credentials)
- Basic understanding of Python and LLMs

### Estimated Time: 15-20 minutes

---
## Step 1: What is MLflow?

**MLflow** is an open-source platform for managing the end-to-end machine learning lifecycle, with specialized support for GenAI and LLM applications.

### Why MLflow for GenAI?

1. **Open Source & Vendor-Neutral**: No lock-in to specific cloud providers or LLM vendors
2. **OpenTelemetry Compatible**: Industry-standard observability that integrates with your existing stack
3. **All-in-One Platform**: Unified solution for tracking, tracing, evaluation, and deployment
4. **30+ Framework Integrations**: Works with OpenAI, LangChain, LlamaIndex, DSPy, AutoGen, and more

### Core Components for GenAI

```
┌─────────────────────────────────────────────────────────┐
│                    MLflow Platform                      │
├─────────────────────────────────────────────────────────┤
│                                                         │
│  📊 Experiment Tracking                                 │
│     Track parameters, metrics, and artifacts            │
│                                                         │
│  🔍 Tracing (Observability)                             │
│     Capture LLM calls, prompts, and tool usage          │
│                                                         │
│  📝 Prompt Management                                   │
│     Version control for prompts and templates           │
│                                                         │
│  ✅ Evaluation                                          │
│     LLM-as-judge metrics and custom scorers             │
│                                                         │
│  🚀 Model Registry                                      │
│     Versioning and deployment of models                 │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

### Architecture Overview

```
┌──────────────┐       ┌──────────────┐       ┌──────────────┐
│   Your App   │──────▶│ MLflow SDK   │──────▶│ Tracking     │
│  (Python)    │       │              │       │ Server       │
└──────────────┘       └──────────────┘       └──────┬───────┘
                                                      │
                                        ┌─────────────┴─────────────┐
                                        │                           │
                                        ▼                           ▼
                                ┌──────────────┐          ┌──────────────┐
                                │  Backend     │          │  Artifact    │
                                │  Store       │          │  Store       │
                                │ (metadata)   │          │ (files)      │
                                └──────────────┘          └──────────────┘
```

---
## Step 2: Installation

Let's install MLflow and the required dependencies for working with LLMs.

In [5]:
# Install MLflow and dependencies
!pip install mlflow>=3.9.0 openai python-dotenv -q

zsh:1: 3.9.0 not found


In [20]:
# Verify installation
import mlflow
print(f"MLflow version: {mlflow.__version__}")

import openai
print(f"OpenAI version: {openai.__version__}")

MLflow version: 3.9.0rc0
OpenAI version: 2.15.0


### Understanding MLflow Versions

- **MLflow 3.x**: Latest release with enhanced tracing and evaluation (recommended)

✅ **You should see MLflow version 3.9.0 or higher**

---
## Step 3: Environment Configuration

Configure your API keys and MLflow settings. We'll use a `.env` file for secure credential management.

In [21]:
# Create a .env file for your API keys (do this once)
# This file should NOT be committed to version control

import os
from pathlib import Path

env_file = Path(".env")

if not env_file.exists():
    print("Creating .env file...")
    with open(env_file, "w") as f:
        f.write("# MLflow GenAI Tutorial Configuration\n")
        f.write("OPENAI_API_KEY=your-api-key-here\n")
        f.write("\n# MLflow Configuration\n")
        f.write("MLFLOW_TRACKING_URI=http://localhost:5000\n")
    print("✅ Created .env file. Please edit it and add your OpenAI API key.")
else:
    print("✅ .env file already exists.")

✅ .env file already exists.


In [22]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv()

# Verify if using Databricks
use_databricks = os.getenv("USE_DATABRICKS_CLIENT") == "True"
print(f"use_databricks: {use_databricks}")
if use_databricks:
    print("✅ Using Databricks hosted foundational models")
    print("✅ Using Databricks workspace client and Databricks profile")
else:
    api_key = os.getenv("OPENAI_API_KEY")
    # Verify API key is set (without displaying it)
    if api_key and api_key != "your-api-key-here":
        print("✅ OpenAI API key is configured")
    else:
        print("❌ Please set your OPENAI_API_KEY in the .env file")

use_databricks: True
✅ Using Databricks hosted foundational models
✅ Using Databricks workspace client and Databricks profile


### 🔒 Security Best Practice

**Always use environment variables or secure credential managers for API keys!**

Never hardcode credentials in your notebooks or commit them to version control.

---
## Step 4: MLflow Tracking Setup

MLflow stores experiment data in two places:
1. **Backend Store**: Metadata (parameters, metrics, tags)
2. **Artifact Store**: Files (models, plots, logs)

For local development, we'll use the filesystem for both.

In [23]:
# Set tracking URI for sqlite backend
tracking_uri = "http://localhost:5000"
mlflow.set_tracking_uri(tracking_uri)

print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"\nData will be stored in: {Path(tracking_uri).absolute()}")

MLflow Tracking URI: http://localhost:5000

Data will be stored in: /Users/jules/git-repos/mlflow-misc/tutorials/http:/localhost:5000


In [24]:
# Create a test experiment
experiment_name = "00-setup-verification"
mlflow.set_experiment(experiment_name)

print(f"✅ Created experiment: {experiment_name}")
print(f"   Experiment ID: {mlflow.get_experiment_by_name(experiment_name).experiment_id}")

2026/01/26 20:17:54 INFO mlflow.tracking.fluent: Experiment with name '00-setup-verification' does not exist. Creating a new experiment.


✅ Created experiment: 00-setup-verification
   Experiment ID: 1


---
## Step 5: First MLflow Run

Let's create your first MLflow run to verify everything is working!

In [25]:
import time

# Start a run
with mlflow.start_run(run_name="setup-test") as run:
    # Log parameters
    mlflow.log_param("test_param", "hello-mlflow")
    mlflow.log_param("framework", "openai")
    
    # Log metrics
    mlflow.log_metric("setup_success", 1.0)
    mlflow.log_metric("timestamp", time.time())
    
    # Log a text artifact
    mlflow.log_text("MLflow GenAI Tutorial - Setup Complete!", "welcome.txt")
    
    print("✅ Run completed successfully!")
    print(f"   Run ID: {run.info.run_id}")
    print(f"   Experiment ID: {run.info.experiment_id}")

✅ Run completed successfully!
   Run ID: b0d2f3dfcebe4b5bb069af3d8faf91df
   Experiment ID: 1
🏃 View run setup-test at: http://localhost:5000/#/experiments/1/runs/b0d2f3dfcebe4b5bb069af3d8faf91df
🧪 View experiment at: http://localhost:5000/#/experiments/1


### 🎉 Congratulations!

You've successfully:
- ✅ Installed MLflow
- ✅ Configured your environment
- ✅ Created an experiment
- ✅ Logged your first run

The data is now stored in the `mlruns` directory.

---
## Step 6: Starting the MLflow UI

The MLflow UI is a web-based interface for visualizing and comparing experiments.

### To start the UI:

**Option 1: From this notebook**
```python
# This will block the cell - run in a separate terminal instead
!mlflow ui --port 5000
```

**Option 2: From terminal (recommended)**
```bash
cd /path/to/this/directory
mlflow ui --port 5000
```

Then open your browser to: **http://localhost:5000**

In [ ]:
# Display instructions for starting the UI
print("""
╔══════════════════════════════════════════════════════════╗
║         MLflow UI - Getting Started                     ║
╚══════════════════════════════════════════════════════════╝

To start the MLflow UI, open a NEW terminal and run:

    cd {}
    mlflow ui --port 5000

Then open your browser to:

    http://localhost:5000

You should see:
  📊 Your experiments listed on the left
  🔍 Runs with parameters and metrics
  📈 Visualization capabilities

Keep the terminal running while using the UI.
Press Ctrl+C in the terminal to stop the UI server.
""".format(Path.cwd()))

---
## Step 7: Exploring the MLflow UI

Once you have the UI running, explore these sections:

### 1. Experiments View
- Lists all your experiments
- Shows experiment metadata and tags
- Search and filter capabilities

### 2. Runs View
- Table of all runs in an experiment
- Columns: parameters, metrics, tags, artifacts
- Compare multiple runs
- Search and filter runs

### 3. Run Details
- Complete information about a single run
- Parameters, metrics, tags
- Artifacts browser
- Model information

### 4. Traces Tab 
- Visualize LLM execution traces
- Timeline view of spans
- Input/output inspection
- Performance analysis

---
## Step 8: Verification Checklist

Let's verify your setup is complete.

In [26]:
# Verification script
def verify_setup():
    checks = []
    
    # Check 1: MLflow installed
    try:
        import mlflow
        checks.append(("✅", "MLflow installed", mlflow.__version__))
    except ImportError:
        checks.append(("❌", "MLflow not installed", "N/A"))
    
    # Check 2: OpenAI installed
    try:
        import openai
        checks.append(("✅", "OpenAI SDK installed", openai.__version__))
    except ImportError:
        checks.append(("❌", "OpenAI SDK not installed", "N/A"))
    
    # Check 3: API key configured
    use_databricks = os.getenv("USE_DATABRICKS_CLIENT") == "True"
    if use_databricks:
        print("✅ Using Databricks Workspace and Databricks profile")
        print("✅ Using Databricks hosted foundational models")
        print("✅ Using Databricks workspace client")
    else:
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key and api_key != "your-api-key-here":
            checks.append(("✅", "OpenAI API key configured", "***"))
        else:
            checks.append(("❌", "OpenAI API key not configured", "N/A"))
    
    # Check 4: Tracking URI set
    tracking_uri = mlflow.get_tracking_uri()
    checks.append(("✅", "Tracking URI set", tracking_uri))
    
    # Check 5: Test experiment exists
    try:
        exp = mlflow.get_experiment_by_name("00-setup-verification")
        if exp:
            checks.append(("✅", "Test experiment created", exp.experiment_id))
        else:
            checks.append(("❌", "Test experiment not found", "N/A"))
    except Exception as e:
        checks.append(("❌", "Test experiment not found", str(e)))
    
    
    # Print results
    print("\n" + "="*60)
    print("       SETUP VERIFICATION RESULTS")
    print("="*60 + "\n")
    
    for status, check, detail in checks:
        print(f"{status} {check:.<40} {detail}")
    
    print("\n" + "="*60)
    
    # Overall status
    if all(status == "✅" for status, _, _ in checks):
        print("\n🎉 Setup complete! You're ready to proceed to the next notebook.")
    else:
        print("\n⚠️  Some checks failed. Please review the errors above.")

verify_setup()

✅ Using Databricks Workspace and Databricks profile
✅ Using Databricks hosted foundational models
✅ Using Databricks workspace client

       SETUP VERIFICATION RESULTS

✅ MLflow installed........................ 3.9.0rc0
✅ OpenAI SDK installed.................... 2.15.0
✅ Tracking URI set........................ http://localhost:5000
✅ Test experiment created................. 1


🎉 Setup complete! You're ready to proceed to the next notebook.


---
## Step 9: Quick Reference - Common Commands

Here are some common MLflow commands you'll use throughout the tutorials:

In [ ]:
# Quick reference guide
print("""
╔══════════════════════════════════════════════════════════════╗
║           MLflow Quick Reference Guide                       ║
╚══════════════════════════════════════════════════════════════╝

📊 EXPERIMENTS
  mlflow.set_experiment("experiment-name")
  mlflow.get_experiment_by_name("experiment-name")
  mlflow.list_experiments()

🏃 RUNS
  with mlflow.start_run(run_name="my-run"):
      # Your code here
      pass

📝 LOGGING
  mlflow.log_param("key", "value")              # Single value
  mlflow.log_params({"key1": "val1", ...})      # Multiple values
  mlflow.log_metric("accuracy", 0.95)           # Single metric
  mlflow.log_metrics({"acc": 0.95, ...})        # Multiple metrics
  mlflow.log_text(text, "file.txt")             # Text artifact
  mlflow.log_artifact("path/to/file")           # File artifact

🔍 TRACING (GenAI)
  mlflow.openai.autolog()                       # Auto-trace OpenAI
  mlflow.langchain.autolog()                    # Auto-trace LangChain
  @mlflow.trace                                 # Manual tracing decorator

🖥️  UI COMMANDS
  mlflow ui                                     # Start UI (port 5000)
  mlflow ui --port 8080                         # Custom port
  mlflow ui --backend-store-uri                 # Custom backend
  mlflow server --backend-store-uri sqlite:///mlflow.db --port 5000

🔧 CONFIGURATION
  mlflow.set_tracking_uri("uri")                # Set tracking location
  mlflow.get_tracking_uri()                     # Get current URI
  mlflow.set_registry_uri("uri")                # Set model registry

""")


╔══════════════════════════════════════════════════════════════╗
║           MLflow Quick Reference Guide                       ║
╚══════════════════════════════════════════════════════════════╝

📊 EXPERIMENTS
  mlflow.set_experiment("experiment-name")
  mlflow.get_experiment_by_name("experiment-name")
  mlflow.list_experiments()

🏃 RUNS
  with mlflow.start_run(run_name="my-run"):
      # Your code here
      pass

📝 LOGGING
  mlflow.log_param("key", "value")              # Single value
  mlflow.log_params({"key1": "val1", ...})      # Multiple values
  mlflow.log_metric("accuracy", 0.95)           # Single metric
  mlflow.log_metrics({"acc": 0.95, ...})        # Multiple metrics
  mlflow.log_text(text, "file.txt")             # Text artifact
  mlflow.log_artifact("path/to/file")           # File artifact

🔍 TRACING (GenAI)
  mlflow.openai.autolog()                       # Auto-trace OpenAI
  mlflow.langchain.autolog()                    # Auto-trace LangChain
  @mlflow.trace         

---
## Summary

In this notebook, you:

1. ✅ Learned what MLflow is and why it's valuable for GenAI
2. ✅ Understood the MLflow architecture and core components
3. ✅ Installed MLflow and required dependencies
4. ✅ Configured your environment with API keys
5. ✅ Set up MLflow tracking
6. ✅ Created your first experiment and run
7. ✅ Learned how to start the MLflow UI
8. ✅ Verified your complete setup

### Next Steps

You're now ready to move on to:

**📓 Notebook 1.2: Experiment Tracking for LLMs**
- Learn how to track LLM experiments
- Log parameters, metrics, and artifacts
- Compare different LLM configurations
- Organize your GenAI experiments

### Resources

- 📚 [MLflow Documentation](https://mlflow.org/docs/latest/)
- 🎯 [MLflow GenAI Guide](https://mlflow.org/docs/latest/genai/)
- 💬 [MLflow Community Slack](https://mlflow.org/community/)
- 🐙 [MLflow GitHub](https://github.com/mlflow/mlflow)

---
## Troubleshooting

### Common Issues

**Problem: MLflow UI won't start**
```bash
# Solution: Check if port 5000 is already in use
mlflow ui --port 5001
```

**Problem: OpenAI API key not recognized**
```python
# Solution: Restart the notebook kernel after adding to .env
# Or manually set:
import os
os.environ["OPENAI_API_KEY"] = "your-key-here"
```

**Problem: Module not found errors**
```bash
# Solution: Ensure you're using the correct Python environment
pip install mlflow openai python-dotenv --upgrade
```